In which I attempt to control multiple robots simultaneously!


In [8]:
# Foundational Sphero API, forked from Simon's Library
import sphero

In [9]:
# Tracking code
import cv2
# SpheroNav base classes
from tracker.trackingfilter import FilterSpheroBlueCover, FilterSpheroYellowCover, FilterGlow

from tracker.traceable import TraceableObject
from tracker.trackerbase import ColorTracker
from util import Vector2D

In [10]:
# Support notebook usage of SpheroTeam
import sys
sys.path.append('../')

In [11]:
import SpheroTeam
# If project is rearchitected, rethink how to make functions accessible
# Look into python module / api design
from SpheroTeam.teamutil import readJsonFile, normalize_angle, angle_between_points
from SpheroTeam.navigation import calibrate_bot_direction, get_bot_position, get_team_offsets

from SpheroTeam.formations import roll_polygon, roll_push
from SpheroTeam.director import bot_go_to_point, team_go_to_points, team_go_to_paths


In [12]:
# General Python Imports
import time
import os
import logging

#### SpheroTeam Setup


For more details about what each line does, refer to the previous notebook (6)


In [14]:
# This is a list of the colored "initial names" of the spheros you want to work with in this session
ACTIVEBOTS = ['RPB', 
              'RWR', 
              'ORG',
             'YYP', 
             'RYR',
             'YPR',
             'GRY']

# ACTIVEBOTS = ['RPB', 
#              'RYR',
#              'RWR',
#              'ORG']

# ACTIVEBOTS = ['YPR']
              
#              'RYR',
#              'RWR',
#              'ORG']

In [15]:
from SpheroTeam import initialize

In [16]:
PROJ_ROOT = os.pardir
CONFIG = os.path.join(PROJ_ROOT, "config.json")
config = readJsonFile(CONFIG)
TEAM_COLORS = config['teamColors']

# Initialize to manage sphero objects
manager = sphero.SpheroManager()

# Initialize Sphero manager using options provided in the config file
manager= initialize.load_sphero_roster(manager, CONFIG)

# Connect to robots
bots = initialize.connect_sphero_team(manager, ACTIVEBOTS)

Sphero-RPB Try #0
Connected!
Sphero-RWR Try #0
Connected!
Sphero-ORG Try #0
Connected!
Sphero-YYP Try #0
IOError Occurred. Trying to connect again.
Sphero-YYP Try #1
IOError Occurred. Trying to connect again.
Sphero-YYP Try #2
Connected!
Sphero-RYR Try #0
Connected!
Sphero-YPR Try #0
Connected!
Sphero-GRY Try #0
Connected!


In [17]:
# Check power, robots shut down after dropping below 7 volts
SpheroTeam.print_team_status(bots)



NOTE! Removes wrong byte in start of header! Byte:  117
NOTE! Removes wrong byte in start of header! Byte:  62
Sphero-RPB:Battery OK | 8.27 V
Sphero-RWR:Battery OK | 8.25 V
NOTE! Removes wrong byte in start of header! Byte:  117
NOTE! Removes wrong byte in start of header! Byte:  62
Sphero-ORG:Battery OK | 8.22 V
NOTE! Removes wrong byte in start of header! Byte:  117
NOTE! Removes wrong byte in start of header! Byte:  62
Sphero-YYP:Battery OK | 8.27 V
Sphero-RYR:Battery OK | 8.23 V
Sphero-YPR:Battery OK | 8.17 V
Sphero-GRY:Battery OK | 7.79 V


In [18]:
TEAM_COLORS = config['teamColors']
TEAM_COLORS


[[255, 0, 0],
 [0, 255, 0],
 [0, 0, 255],
 [255, 0, 255],
 [255, 255, 0],
 [255, 133, 0],
 [255, 192, 203]]

In [36]:
SpheroTeam.set_team_colors(bots, TEAM_COLORS)


#### Camera Tracker Setup



In [20]:
tracker = ColorTracker(config['cameraID'])

In [21]:
imageX, imageY = tracker.image_size

print "Image: {} x {} pixels".format(imageX, imageY)

Image: 640 x 480 pixels


In [22]:
# For now, blue is the only color we can reliably track
traceable_color = [0, 0, 255]  # RGB Blue
traceable_blue = TraceableObject("BLUE")
traceable_blue.filter = FilterSpheroBlueCover()

In [102]:
# make sure everyone's light is off before calibrating.

for bot in bots:
    bot.set_rgb(0,0,0)
    
    
    

In [103]:
# Blue seems to be the most reliable color to track with.
# Be suspicious if any of the offsets are 0!!!
# on reflection, may want method to be able to redo 


# the offsets of just a few select robots

offsets = get_team_offsets(bots, traceable_blue, traceable_color, tracker)

Start (328,204)
End   (350,256)
Angle 67.0678995624
'q' to quit, else calibrate next robot a
Start (446,170)
End   (520,109)
Angle 320.50041147
'q' to quit, else calibrate next robot a
Start (405,169)
End   (375,252)
Angle 109.872175819
'q' to quit, else calibrate next robot a
Start (286,253)
End   (214,197)
Angle 217.874983651
'q' to quit, else calibrate next robot a
Start (309,160)
End   (338,132)
Angle 316.005086005
'q' to quit, else calibrate next robot a
Start (300,89)
End   (309,35)
Angle 279.462322208
'q' to quit, else calibrate next robot a
Start (235,213)
End   (213,269)
Angle 111.447736327
'q' to quit, else calibrate next robot a


In [121]:
# to redo individual offsets, do this in-place modification of the offset
def team_redo_offset(offsets, bots, index, 
                traceable_object, traceable_color, tracker):
    """
        If some of the offsets are bad, redo the offset for bot at index
        i without losing the others
    """
    offsets[index] = calibrate_bot_direction(bots[index], traceable_object, 
                                             traceable_color, tracker)
    
    return offsets

In [122]:
offsets = team_redo_offset(offsets, bots, 1, traceable_blue, traceable_color, tracker)

In [7]:
?roll_polygon

In [99]:
# Roll in a square to show it's working
# can roll in single direction using roll-push.
# roll_polygon(bots, 3, offsets, 0, speed=70, TIMEOUT=1000)


Dispatch Time 0.5
Dispatch Time 0.40299987793


In [127]:
for i, bot in enumerate(bots):
#     offsets = team_redo_offset(offsets, bots, i, traceable_blue, traceable_color, tracker)
    
    SpheroTeam.roll_sphero(bot, 60, 0, offsets[i])
    

    
    
    time.sleep(1.5)
    

Exception in thread SpheroReceiverThread:
Traceback (most recent call last):
  File "C:\Users\cyick\Anaconda2\lib\threading.py", line 801, in __bootstrap_inner
    self.run()
  File "C:\Users\cyick\Anaconda2\lib\threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "C:\Users\cyick\AppData\Roaming\Python\Python27\site-packages\sphero\core.py", line 422, in _receiver
    header = self._receive_header()
  File "C:\Users\cyick\AppData\Roaming\Python\Python27\site-packages\sphero\core.py", line 306, in _receive_header
    first_byte = struct.unpack('B', raw_data)[0]
error: unpack requires a string argument of length 1

Exception in thread SpheroReceiverThread:
Traceback (most recent call last):
  File "C:\Users\cyick\Anaconda2\lib\threading.py", line 801, in __bootstrap_inner
    self.run()
  File "C:\Users\cyick\Anaconda2\lib\threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "C:\Users\cyick\AppData\Roaming\Python\Pytho

In [98]:
roll_polygon(bots, 1, offsets, 180, speed=60, TIMEOUT=1500)



Dispatch Time 0.482000112534


### Finally: Group sending of robots to points




One problem: when robot rolls off the viewing field. Maybe when a robot cannot be seen, it should roll in the opposite of the last angle it rolled in.


In [15]:
?get_bot_position()

In [ ]:
print imageX
print imageY

In [51]:
# Every robot, after some pushing activity, will use this path to skirt the perimeter
recoveryPaths = [
    [
        [320, 400],  # 
        [100, 245]
    ],
    
    [
        [320, 100],
        [100, 235]
    ],
    
    [
        [320, 400],  # 
        [100, 250]
    ],
    
    [
        [320, 100],
        [100, 230]
    ],
    
    [
        [320, 400],  # 
        [100, 255]
    ],
    [
        
        [320, 100],
        [100, 225]
    ],
    [
        [320, 100],
        [100, 225]
    ]
]

In [7]:
# Debugging individual robot navigation
# team_go_to_points([bots[2]], [[50, 50]], [offsets[2]], traceable_blue, traceable_color, stopRadius=15, )

In [29]:
from SpheroTeam.navigation import get_rectangle_targets

ImportError: cannot import name get_rectangle_targets

In [32]:
# Send robots to a cluster around 300, 150
targets = [ [300, 150 + i*5] for i in range(len(bots))] # space out the robots along x = 100

# targets = get_rectangle

team_go_to_points(bots, targets, offsets, traceable_blue, traceable_color, imageX, imageY, tracker, stopRadius=15, )



Bot Sphero-RPB
Go 397,339 to 300,150| Distance 212.438226315 / -117.168142371
Stopped at 307,162, with dist 13.8924439894
Bot Sphero-RWR
Go 407,166 to 300,155| Distance 107.563934476 / -174.130399557
Stopped at 315,138, with dist 22.6715680975
Bot Sphero-ORG
Go 359,270 to 300,160| Distance 124.823875921 / -118.207490879
Stopped at 292,132, with dist 29.1204395571
Bot Sphero-YYP
Go 419,300 to 300,165| Distance 179.961106909 / -131.395579461
Stopped at 321,156, with dist 22.8473193176
Bot Sphero-RYR
Go 342,221 to 300,170| Distance 66.0681466366 / -129.472459848
Stopped at 294,180, with dist 11.6619037897
Bot Sphero-YPR
Go 359,223 to 300,175| Distance 76.0591874792 / -140.869600443
Stopped at 283,173, with dist 17.1172427686
Bot Sphero-GRY
Go 370,212 to 300,180| Distance 76.9675256196 / -155.432828679
Stopped at 282,190, with dist 20.591260282


### Now, move them together




In [28]:
SpheroTeam.set_team_colors(bots, TEAM_COLORS)    

In [29]:
# Test a pentagon
roll_polygon(bots, 5, offsets, 180, speed=60, TIMEOUT=1200)

Dispatch Time 0.162999868393
Dispatch Time 0.181999921799
Dispatch Time 0.164999961853
Dispatch Time 0.197000026703
Dispatch Time 0.164000034332


In [61]:
# Bring a lost sphero back if it rolled away (consider making an arena wall in future)
# SpheroTeam.roll_sphero(bots[0], 40, 180, offsets[0])

In [ ]:
# Roll all team along angle 0 at a speed of 70 for 3 seconds
roll_push(bots, 0, 70, offsets, TIMEOUT=3000)

In [30]:
# Show how robots would recover from "overshooting" the target by going to the outside of the arena
#, and then returning to a home area.
team_go_to_paths(bots, recoveryPaths, offsets, traceable_blue, traceable_color)

Bot Sphero-RPB
Get from 248,130 to 320,400| Distance 279.435144533 / 75.0685828219
Stopped at 358,408, with dist 38.8329756779
Get from 355,387 to 100,245| Distance 291.871547089 / -150.888195474
Stopped at 135,253, with dist 35.902646142
Bot Sphero-RWR
Get from 134,236 to 320,100| Distance 230.417013261 / -36.1736290458
Stopped at 319,82, with dist 18.0277563773
Get from 317,87 to 100,235| Distance 262.665186121 / 145.70493089
Stopped at 103,219, with dist 16.2788205961
Bot Sphero-ORG
Get from 90,237 to 320,400| Distance 281.902465402 / 35.3250346381
Stopped at 302,417, with dist 24.7588368063
Get from 326,432 to 100,250| Distance 290.172362571 / -141.155168343
Stopped at 92,232, with dist 19.6977156036


Exception in thread SpheroReceiverThread:
Traceback (most recent call last):
  File "C:\Users\cyick\Anaconda2\lib\threading.py", line 801, in __bootstrap_inner
    self.run()
  File "C:\Users\cyick\Anaconda2\lib\threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "C:\Users\cyick\AppData\Roaming\Python\Python27\site-packages\sphero\core.py", line 422, in _receiver
    header = self._receive_header()
  File "C:\Users\cyick\AppData\Roaming\Python\Python27\site-packages\sphero\core.py", line 306, in _receive_header
    first_byte = struct.unpack('B', raw_data)[0]
error: unpack requires a string argument of length 1

Exception in thread SpheroReceiverThread:
Traceback (most recent call last):
  File "C:\Users\cyick\Anaconda2\lib\threading.py", line 801, in __bootstrap_inner
    self.run()
  File "C:\Users\cyick\Anaconda2\lib\threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "C:\Users\cyick\AppData\Roaming\Python\Pytho

Challenges: 

- Path planning is not the same thing as simply choosing to arrive at a point.
    - Need to come up with algorithm to avoid obstacles, knowing that you may miss goal points by 10-20 centimeters.
    
- Tuning the right KP parameter is tricky
- Coming up with recovery paths is tricky

- How to stop robots from bumping into each other


- When robot runs out of bounds, we can't see it... need to create a "fence" around the arena and that would fix it.

- Tracker can only reliably "see" 1 color at once, so workaround is only using that 1 color for tracking.

Alternate ideas that weren't implemented due to lack of time

- When swarm needs to converge on 1 point: have everyone drive at fixed speed

- drive for however long that distance takes in pixels

- Let everyone drive simultaneously by giving each robot a blind directive, and not using any feedback to tune its behavior (aka a "formation")


- Write a function that given a singular (x,y) point, generates a list of pixels that each robot would aim for, along with an "shape" and relevant parameters for how the robots should be arranged (aka box, line, circle, hexagon, etc)

- Write a function that given a singular target (x,y) point and a robot, define a list of points that the robot should visit before reaching the target points

